In [ ]:
import tensorflow as tf
import collections
#from tensorflow import keras
from keras import layers
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout,GlobalMaxPool2D,GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
#inputshape=(height,width,deep)

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import h5py
import os 
from PIL import Image
import sklearn
from sklearn import preprocessing
import scipy
import numpy as np
from keras import backend as K
from keras.models import load_model
import os,sys

In [ ]:
train_dataset = h5py.File("./dataset_oracle_20200602.h5",'r')
X_train = np.array(train_dataset['x_train'][:])
Y_train = np.array(train_dataset['y_train'][:])
X_test = np.array(train_dataset['x_test'][:])
Y_test = np.array(train_dataset['y_test'][:])
train_dataset.close()
#train_data= create_dataset(X_train,Y_train)

In [ ]:
class_names=["fire","soil","wood","water","gas","sky","human","life"]
print(len(X_train))
for i in range(25):
    print(np.squeeze(X_train[i]),np.sum(X_train[i]==1))
    print(class_names[int(Y_train[i])])

In [ ]:
print("number of traning examples = "+str(X_train.shape[0]))
print("number of test examples = "+str(X_test.shape[0]))
print("X_train shape:" + str(X_train.shape))
print("Y_train shape:" + str(Y_train.shape))
print("X_test shape:" + str(X_test.shape))
print("Y_test shape:" + str(Y_test.shape))
train_data=(X_train,Y_train)
test_data=(X_test,Y_test)

In [ ]:
#Build a model
def Pimodel(input_shape):
    X_input = Input(shape=input_shape)
    X=ZeroPadding2D(padding=(1,1))(X_input)
    X=Conv2D(1,kernel_size=(3,6),strides=(3,6),use_bias=None)(X)
    X=Activation('relu')(X)
    #X=Conv2D(1,kernel_size=(2,3),strides=(2,2))(X)
    #X=Activation("sigmoid")(X)
    X=Flatten()(X)
    Y=Activation('softmax')(X)
    model=Model(inputs=X_input,outputs=Y,name="JSmodel")
    return model

In [ ]:
picmodel = Pimodel((10,10,1))

In [ ]:
import keras
picmodel.compile(optimizer=keras.optimizers.Adam(lr=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-08,decay=0.0),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
picmodel.summary()

In [ ]:
history=picmodel.fit(x=X_train,y=Y_train,batch_size=200,epochs=1000,validation_data=(X_test,Y_test))

In [ ]:
preds = picmodel.evaluate(x=X_test,y=Y_test)

print('Test Loss=' + str(preds[0]))
print("Test Accuacy="+str(preds[1]))

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(history.history["accuracy"])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.ylim((0, 1))
plt.legend(['train', 'test'], loc='best')

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(history.history["loss"])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')

## Check the weight and bias

In [ ]:
kernel_weights=picmodel.layers[2].get_weights()[0]
#kernel_weights
print(np.squeeze(kernel_weights))

In [ ]:
print(picmodel.summary())
print(picmodel.get_weights())

In [ ]:
## the layer output 

In [ ]:
class_names=["fire","soil","wood","water","gas","sky","human","life"]
picmodel=load_model("./model_orcale_20200602.h5")

In [ ]:
print(picmodel.summary())

In [ ]:
kernel_weights=np.squeeze(picmodel.get_weights())
#print(kernel_weights.flatten())
a1=np.squeeze(kernel_weights)[0:3,0:6].flatten()
print(kernel_weights)
N=abs(a1.flatten())[np.argmax(abs(a1.flatten()))]
print(N)
a1_nor=a1.flatten()/N
print(a1_nor)

In [ ]:
def get_layer_output(model,x,index=0):
    layer = K.function([model.input],[model.layers[index].output])
    return layer([x])[0]

In [ ]:
path="./dataset/fire/"
imlist=os.listdir(path)
fire_t=[]
Fire_t=[]

fn_t=[]
for elef in imlist:
    input_x = np.array(np.load(r"./dataset/fire/"+elef))
    input_x = np.expand_dims(input_x,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_4 = get_layer_output(picmodel,input_x,index=4)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    fire_t.append(class_names[np.argmax(layer_5)])
    #print(elef,layer_2.flatten())
    if class_names[np.argmax(layer_5)]=="fire":
        FN_t=[]
        FN_t.append(elef)
        FN_t.append(layer_2.flatten())
        fn_t.append(FN_t)
        print(elef,layer_2.flatten(),class_names[np.argmax(layer_5)])
        Fire_t.append(elef)
print(fn_t)
f1=collections.Counter(fire_t)
print(len(fire_t),collections.Counter(fire_t))

In [ ]:
path="./dataset/soil/"
imlist=os.listdir(path)
soil_T=[]
Soil_T=[]
for elef in imlist:
    input_x = np.array(np.load(r"./dataset/soil/"+elef))
    input_x = np.expand_dims(input_x,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    #print(class_names[np.argmax(layer_6)])
    soil_T.append(class_names[np.argmax(layer_5)])
    if class_names[np.argmax(layer_5)]=="soil":
        print(elef,layer_5,class_names[np.argmax(layer_5)])
        #print(elef)
        #print(layer_2.flatten())
        #print(class_names[np.argmax(layer_5)])
        Soil_T.append([elef,layer_2.flatten()])
print(len(soil_T))
s1=collections.Counter(soil_T)
print(collections.Counter(soil_T))
print(Soil_T)

In [ ]:
path="./dataset/wood/"
imlist=os.listdir(path)
wood_T=[]
Wood_T=[]
for elef in imlist:
    input_x = np.array(np.load(r"./dataset/wood/"+elef))
    input_x = np.expand_dims(input_x,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    wood_T.append(class_names[np.argmax(layer_5)])
    if class_names[np.argmax(layer_5)]=="wood":
        print(elef,layer_5,class_names[np.argmax(layer_5)])
        #print(elef)
        #print(layer_2.flatten())
        #print(class_names[np.argmax(layer_5)])
        Wood_T.append([elef,layer_2.flatten()])
print(len(wood_T))
print(Wood_T)
W1=collections.Counter(wood_T)
print(collections.Counter(wood_T))  

In [ ]:
path="./dataset/water/"
imlist=os.listdir(path)
water_T=[]
Water_T=[]
for elef in imlist:
    input_x = np.array(np.load(r"./dataset/water/"+elef))
    input_x = np.expand_dims(input_x,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    #print(class_names[np.argmax(layer_6)])
    water_T.append(class_names[np.argmax(layer_5)])
    if class_names[np.argmax(layer_5)]=="water":
        #print(elef,layer_5,class_names[np.argmax(layer_5)])
        print(elef)
        print(layer_2.flatten())
        print(class_names[np.argmax(layer_5)])
        Water_T.append([elef,layer_2.flatten()])
#print(len(water_T))
w1=collections.Counter(water_T)
print(collections.Counter(water_T))
print(Water_T)

In [ ]:
path="./dataset/gas/"
imlist=os.listdir(path)
gas_T=[]
Gas_T=[]
for elef in imlist:
    input_x = np.array(np.load(r"./dataset/gas/"+elef))
    input_x = np.expand_dims(input_x,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    #print(class_names[np.argmax(layer_6)])
    #print(elef,layer_6,np.argmax(layer_6))
    gas_T.append(class_names[np.argmax(layer_5)])
    if class_names[np.argmax(layer_5)]=="gas":
        print(elef,layer_2.flatten(),class_names[np.argmax(layer_5)])
        Gas_T.append([elef,layer_2.flatten()])
print(len(gas_T))
G1=collections.Counter(gas_T)
print(Gas_T)
print(collections.Counter(gas_T))

In [ ]:
path="./dataset/sky/"
imlist=os.listdir(path)
sky_T=[]
Sky_T=[]
for elef in imlist:
    input_x = np.array(np.load(r"./dataset/sky/"+elef))
    input_x = np.expand_dims(input_x,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    #print(class_names[np.argmax(layer_6)])
    #print(elef,layer_6,np.argmax(layer_5))
    sky_T.append(class_names[np.argmax(layer_5)])
    if class_names[np.argmax(layer_5)]=="sky":
        print(elef,layer_2.flatten(),class_names[np.argmax(layer_5)])
        Sky_T.append([elef,layer_2.flatten()])
print(len(sky_T))
S1=collections.Counter(sky_T)
print(Sky_T)
print(collections.Counter(sky_T))

In [ ]:
path="./dataset/human/"
imlist=os.listdir(path)
human_T=[]
Human_t=[]
for elef in imlist:
    input_x = np.array(np.load(r"./dataset/human/"+elef))
    input_x = np.expand_dims(input_x,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    #print(elef,layer_6,np.argmax(layer_6))
    human_T.append(class_names[np.argmax(layer_5)])
    if class_names[np.argmax(layer_5)]=="human":
        print(elef,layer_2.flatten(),class_names[np.argmax(layer_5)])
        Human_t.append([elef,layer_2.flatten()])
H1=collections.Counter(human_T)
print(len(human_T))
print(Human_t)
print(collections.Counter(human_T))

In [ ]:
path="./dataset/life/"
imlist=os.listdir(path)
life_T=[]
Life_T=[]
for elef in imlist:
    input_x = np.array(np.load(r"./dataset/life/"+elef))
    input_x = np.expand_dims(input_x,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    #print(class_names[np.argmax(layer_6)])
    #print(elef,layer_6,class_names[np.argmax(layer_6)])
    life_T.append(class_names[np.argmax(layer_5)])
    if class_names[np.argmax(layer_5)]=="life":
        print(elef,layer_2.flatten(),class_names[np.argmax(layer_5)])
        Life_T.append([elef,layer_2.flatten()])
    
L1=collections.Counter(life_T)
print(len(life_T))
print(Life_T)
print(collections.Counter(life_T))

In [ ]:
import pandas as pd
#dict1 = pd.DataFrame(pd.Series(f1),columns=["numbers"],index=["fire","soil","water","wood","gas","sky","human","life"])
x=["fire","soil","water","wood","gas","sky","human","life"]
dict1=pd.Series(f1).reindex(["fire","soil","water","wood","gas","sky","human","life"])
dict1[np.squeeze(np.argwhere(np.isnan(dict1)))]=0
dict2=pd.Series(s1).reindex(["fire","soil","water","wood","gas","sky","human","life"])
dict2[np.squeeze(np.argwhere(np.isnan(dict2)))]=0
dict3=pd.Series(w1).reindex(["fire","soil","water","wood","gas","sky","human","life"])
dict3[np.squeeze(np.argwhere(np.isnan(dict3)))]=0
dict4=pd.Series(W1).reindex(["fire","soil","water","wood","gas","sky","human","life"])
dict4[np.squeeze(np.argwhere(np.isnan(dict4)))]=0
dict5=pd.Series(G1).reindex(["fire","soil","water","wood","gas","sky","human","life"])
dict5[np.squeeze(np.argwhere(np.isnan(dict5)))]=0
dict6=pd.Series(S1).reindex(["fire","soil","water","wood","gas","sky","human","life"])
dict6[np.squeeze(np.argwhere(np.isnan(dict6)))]=0
dict7=pd.Series(H1).reindex(["fire","soil","water","wood","gas","sky","human","life"])
dict7[np.squeeze(np.argwhere(np.isnan(dict7)))]=0
dict8=pd.Series(L1).reindex(["fire","soil","water","wood","gas","sky","human","life"])
dict8[np.squeeze(np.argwhere(np.isnan(dict8)))]=0
plt.figure(figsize=(24,10))
plt.subplot(241)
plt.bar(x,dict1,color="red")
plt.title("fire regonition")
for i,j in zip(x, dict1):
        plt.text(i, j, str(int(j)), ha='center', va='bottom', fontsize=10.5)
#print(dict1)
plt.subplot(242)
plt.bar(x,dict2,color="yellow")
plt.title("soil regonition")
for i,j in zip(x, dict2):
        plt.text(i, j, str(int(j)), ha='center', va='bottom', fontsize=10.5)
#print(dict2)
plt.subplot(243)
plt.bar(x,dict3,color="brown")
plt.title("wood regonition")
for i,j in zip(x, dict3):
        plt.text(i, j, str(int(j)), ha='center', va='bottom', fontsize=10.5)
#print(dict1)
plt.subplot(244)
plt.bar(x,dict4,color="blue")
plt.title("water regonition")
for i,j in zip(x, dict4):
        plt.text(i, j, str(int(j)), ha='center', va='bottom', fontsize=10.5)
#print(dict1)
plt.subplot(245)
plt.bar(x,dict5,color="green")
plt.title("gas regonition")
for i,j in zip(x, dict5):
        plt.text(i, j, str(int(j)), ha='center', va='bottom', fontsize=10.5)
#print(dict1)
plt.subplot(246)
plt.bar(x,dict6,color="orange")
plt.title("sky regonition")
for i,j in zip(x, dict6):
        plt.text(i, j, str(int(j)), ha='center', va='bottom', fontsize=10.5)
#print(dict1)
plt.subplot(247)
plt.bar(x,dict7,color="black")
plt.title("human regonition")
for i,j in zip(x, dict7):
        plt.text(i, j, str(int(j)), ha='center', va='bottom', fontsize=10.5)
#print(dict1)
plt.subplot(248)
plt.bar(x,dict8,color="purple")
plt.title("life regonition")
for i,j in zip(x, dict8):
        plt.text(i, j, str(int(j)), ha='center', va='bottom', fontsize=10.5)
#print(dict1)
#list1=dict1.values.tolist()
plt.savefig("regonition.png")
plt.show()